In [1]:
%matplotlib notebook

import os
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.spatial import distance_matrix

from tldist.database import get_database
from tldist.similarity.similarity import Similarity
from tldist.data import Data
from tldist.fingerprint import Fingerprint
from tldist.similarity.display import SimilarityDisplay

from astropy.coordinates import SkyCoord
from astropy import units
       
db = get_database('unqlite', 'basic_notebook.db')
similarities = db.find('similarity')
similarity_tsne = Similarity.similarity_factory(similarities[0])

simdisp = SimilarityDisplay(similarity_tsne, db)

INFO     2018-05-09 11:54:52,784 database   __new__    val
INFO     2018-05-09 11:54:52,786 database   __init__   


IndexError: list index out of range

In [ ]:
similarities

In [2]:
db.find('fingerprint')

[]